# Засоби підготовки та налізу даних
## Лабораторна робота №4
### ФБ-22 Лаптєв Денис


#### Підготовка середовища

In [2]:
import pandas as pd
import numpy as np
import timeit
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request
print ('setup complete')

setup complete


##### Завантаження датасету

In [6]:
url = "https://archive.ics.uci.edu/static/public/235/data.csv"
filename = "data/data.csv"

urllib.request.urlretrieve(url, filename)

('data/data.csv', <http.client.HTTPMessage at 0x2e3e2776d50>)

In [39]:
def load_data():
    url = "https://archive.ics.uci.edu/static/public/235/data.csv"
    filename = "data/data.csv"

    urllib.request.urlretrieve(url, filename)

def create_df():
    df = pd.read_csv("data/data.csv")

    df = df.drop(df.loc[df['Sub_metering_1'] =='?'].index)

    df['Global_active_power'] = df['Global_active_power'].astype(float)
    df['Global_reactive_power'] = df['Global_reactive_power'].astype(float)
    df['Voltage'] = df['Voltage'].astype(float)
    df['Global_intensity'] = df['Global_intensity'].astype(float)
    df['Sub_metering_1'] = df['Sub_metering_1'].astype(float)
    df['Sub_metering_2'] = df['Sub_metering_2'].astype(float)
    df['Sub_metering_3'] = df['Sub_metering_3'].astype(float)

    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df = df.drop(['Date', 'Time'], axis=1)

    return df

#load_data()
df = create_df()

#print(df)
#df.to_csv('output.csv', index=False)

C:\Users\Denys\AppData\Local\Temp\ipykernel_11232\4153129972.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_11232\4153129972.py:20: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


In [34]:
df['Time'].dtype

dtype('<M8[ns]')

#### Хід роботи

- проаналізувати часові витрати на виконання процедур(профілювання часу виконання)
- зробити висновки щодо ситуацій, в яких має сенс віддати перевагу тій чи іншій структурі даних. Висновки оформити звітом із зазначеним часом виконання та оцінкою по 5-бальній шкалі зручності виконання операцій відбору.
- залишити лише ті спостереження, в яких немає порожніх спостережень (порожні значення – пусті поля між роздільником – ? – 28.04.2007, як приклад).

1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [ ]:
def fun_1_pd(df):
    result_df = df[(df['Global_active_power'] >= 5)]
    return result_df

def fun_1_np(df):
    result_df = df
    return result_df


2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [ ]:
def fun_2_pd(df):
    result_df = df[(df['Voltage'] >= 235)]
    return result_df

3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [48]:
def fun_3_pd(df):
    result_df = df[(df['Global_intensity'].between(19, 20)) and (df['Sub_metering_2'] > df['Sub_metering_3'])]
    return result_df

4. Обрати випадковим чином 5000(00)? домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії.

In [ ]:
def fun_4_np(df):
    result_df = df[''].np.random.choice(replace=True)
    return result_df

5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.